# 填充
在应用多层卷积时，我们常常丢失边缘像素。 由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素。但随着我们应用许多连续卷积层，累积丢失的像素数就多了。

解决这个问题的简单方法即为填充（padding）:在输入图像的边界填充元素（通常填充元素是）。我们将3x3输入填充到5x5，那么它的输出就增加为4x4。阴影部分是第一个输出元素以及用于输出计算的输入和核张量元素：

<img src="pic\13.jpg"  width="500"/>

通常，如果我们添加行填充（大约一半在顶部，一半在底部）和列填充（左侧大约一半，右侧一半），则输出形状将为

<img src="pic\14.jpg"  width="500"/>

卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。 选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。

此外，使用奇数的核大小和填充大小也提供了书写上的便利。对于任何二维张量X，当满足： 
1. 卷积核的大小是奇数； 
2. 所有边的填充行数和列数相同； 
3. 输出与输入具有相同高度和宽度 则可以得出：输出Y[i, j]是通过以输入X[i, j]为中心，与卷积核进行互相关计算得到的。

In [6]:
import torch
from torch import nn

# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([1, 1, 8, 8])


torch.Size([8, 8])

当卷积核的高度和宽度不同时，我们可以填充不同的高度和宽度，使输出和输入具有相同的高度和宽度。在如下示例中，我们使用高度为5，宽度为3的卷积核，高度和宽度两边的填充分别为2和1。

In [7]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([1, 1, 8, 8])


torch.Size([8, 8])

# 步幅
在前面的例子中，我们默认每次滑动一个元素。 但是，有时候为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素。

们将每次滑动元素的数量称为步幅（stride）。到目前为止，我们只使用过高度或宽度为的步幅，那么如何使用较大的步幅呢？下图是垂直步幅为3，水平步幅为2的二维相关运算。

可以看到，为了计算输出中第一列的第二个元素和第一行的第二个元素，卷积窗口分别向下滑动三行和向右滑动两列。但是，当卷积窗口继续向右滑动两列时，没有输出，因为输入元素无法填充窗口（除非我们添加另一列填充）。

<img src="pic\15.jpg"  width="500"/>

通常，当垂直步幅为Sh、水平步幅为Sw，输出形状为

<img src="pic\16.jpg" width="500"/>

我们将高度和宽度的步幅设置为2，从而将输入的高度和宽度减半。

In [9]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)

X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([1, 1, 4, 4])


torch.Size([4, 4])

In [10]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([1, 1, 2, 2])


torch.Size([2, 2])

默认情况下，填充为0，步幅为1。在实践中，我们很少使用不一致的步幅或填充，也就是说，我们通常**Ph=Pw**有和**Sh=Sw**。